In [2]:
import os
import numpy as np
import h5py
from dateutil.parser import parse
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs.layout import YAxis, XAxis, Margin
from plotly.subplots import make_subplots

In [3]:
class MasterKeys:
    data_type = 'data_type'
    l_over_d = 'l_over_d'
    data_path = 'data_path'
    nexus_path = 'nexus_path'
    start_time = 'start_time'
    start_time_friendly_display = 'start_time_friendly_display'
    start_time_in_seconds = 'start_time_in_seconds'

class DataType:
    ob = 'ob'
    dc = 'dc'
    ni = 'Ni powder'

class PV:
    bm_averaged_count_rate = "BL10:Det:BM1:Histo:Daq:Rate"    
    averaged_count_rate_adted = "BL10:Det:T1:CountRate_RBV"
    averaged_count_rate_serval = "BL10:CS:tpx3:cam1:PelEvtRate_RBV"
    # gdc_serval_chip0 = "BL10:Det2:tpx3:LastGDCChipZero"
    # gdc_serval_chip1 = "BL10:Det2:tpx3:LastGDCChipOne"
    # gdc_serval_chip2 = "BL10:Det2:tpx3:LastGDCChipTwo"  
    # gdc_serval_chip3 = "BL10:Det2:tpx3:LastGDCChipThree" 

class PVnames:
    title = "Average Count Rates"
    bm_averaged_count_rate = "Beam Monitor"    
    averaged_count_rate_adted = "ADtED"
    averaged_count_rate_serval = "Serval"

class PVtags:
    mean = "mean"
    min = "min"
    max = "max"
    
IPTS = "IPTS-34969"
NEXUS_PATH = os.path.join("/SNS", "VENUS", IPTS, 'nexus')

In [4]:
assert os.path.exists(NEXUS_PATH)

# Retrieve PV values

In [5]:
def fill_master_dict(from_run_number, to_run_number, data_type, l_over_d, data_path):
    master_dict = {}
    for run_number in range(from_run_number, to_run_number + 1):
        master_dict[run_number] = {
            MasterKeys.data_type: data_type,
            MasterKeys.l_over_d: l_over_d,
            MasterKeys.data_path: data_path,
            MasterKeys.nexus_path: os.path.join(NEXUS_PATH, f"VENUS_{run_number}.nxs.h5"),
            MasterKeys.start_time: None,
            MasterKeys.start_time_friendly_display: None,
            MasterKeys.start_time_in_seconds: None,
            PV.bm_averaged_count_rate: {PVtags.mean: None,
                                        PVtags.min: None,
                                        PVtags.max: None},
            PV.averaged_count_rate_adted: {PVtags.mean: None,
                                        PVtags.min: None,
                                        PVtags.max: None},
            PV.averaged_count_rate_serval: {PVtags.mean: None,
                                        PVtags.min: None,
                                        PVtags.max: None},
            # PV.gdc_serval_chip0: None,
            # PV.gdc_serval_chip1: None,
            # PV.gdc_serval_chip2: None,
            # PV.gdc_serval_chip3: None,
        }
    return master_dict

In [6]:
def get_start_time_friendly_display(start_time):
    _time = parse(start_time)
    month = _time.month
    day = _time.day
    hour = _time.hour
    minute = _time.minute
    second = _time.second
    return f"{month}/{day} {hour}:{minute}:{second}"

def convert_to_seconds(start_time):
    _time = parse(start_time)
    return int(_time.timestamp())
    



In [7]:
def retrieve_metadata(master_dict):

    list_pvs = [
        PV.bm_averaged_count_rate,
        PV.averaged_count_rate_adted,
        PV.averaged_count_rate_serval,
        # PV.gdc_serval_chip0,
        # PV.gdc_serval_chip1,
        # PV.gdc_serval_chip2,
        # PV.gdc_serval_chip3
    ]

    for run_number, run_dict in master_dict.items():
        nexus_path = run_dict[MasterKeys.nexus_path]
        if not os.path.exists(nexus_path):
            print(f"Missing {nexus_path}")
            continue
        try:
            with h5py.File(nexus_path, 'r') as hdf5_data:
                for pv in list_pvs:
                    master_dict[run_number][pv] = {PVtags.mean: np.mean(hdf5_data['entry']['DASlogs'][pv]['value'][:]),
                                                   PVtags.min: np.min(hdf5_data['entry']['DASlogs'][pv]['value'][:]),
                                                   PVtags.max: np.max(hdf5_data['entry']['DASlogs'][pv]['value'][:]),
                                                   }
                    master_dict[run_number][MasterKeys.start_time] = hdf5_data['entry']['start_time'][0].decode("utf8")
                    master_dict[run_number][MasterKeys.start_time_friendly_display] = get_start_time_friendly_display(hdf5_data['entry']['start_time'][0].decode("utf8"))
                    master_dict[run_number][MasterKeys.start_time_in_seconds] = convert_to_seconds(hdf5_data['entry']['start_time'][0].decode("utf8"))

        except:
            pass



Init dictionary according to excel spreadsheet

In [8]:
master_dict = {}

# first row
master_dict_first_row = fill_master_dict(6690, 6712, DataType.ob, 400, "/IPTS-34969/images/tpx3/raw/radiographs/February20_2025_Ni_5C_MCP_TPX3_0_7AngsMin/")
master_dict.update(master_dict_first_row)

# second row
master_dict_second_row = fill_master_dict(6713, 6715, DataType.dc, None, "/IPTS-34969/images/tpx3/raw/df/February21_2025_OB_5C_MCP_TPX3_0_7AngsMin/")
master_dict.update(master_dict_second_row)

# third row
master_dict_third_row = fill_master_dict(6718, 6729, DataType.ni, 400, "/IPTS-34969/images/tpx3/raw/df/February21_2025_Ni_powder_5_0C_MCP_TPX3_0_7AngsMin/")
master_dict.update(master_dict_third_row)

# fourth row
master_dict_fourth_row = fill_master_dict(6730, 6743, DataType.ni, 800, "/IPTS-34969/images/tpx3/raw/df/February22_2025_Ni_powder_5_0C_MCP_TPX3_0_7AngsMin/")
master_dict.update(master_dict_fourth_row)

# fifth row
master_dict_fifth_row = fill_master_dict(6744, 6749, DataType.ni, 800, "/IPTS-34969/images/tpx3/raw/df/February22_2025_Ni_powder_5_0C_MCP_TPX3_2_0AngsMin/")
master_dict.update(master_dict_fifth_row)

# sixth row
master_dict_sixth_row = fill_master_dict(6750, 6762, DataType.ob, 800, "/IPTS-34969/images/tpx3/raw/ob/February23_2025_OB_5_0C_MCP_TPX3_2_0AngsMin/")    
master_dict.update(master_dict_sixth_row)

# seventh row
master_dict_seventh_row = fill_master_dict(6763, 6770, DataType.ob, 400, "/IPTS-34969/images/tpx3/raw/ob/February24_2025_OB_5_0C_MCP_TPX3_2_0AngsMin/")
master_dict.update(master_dict_seventh_row)

# eighth row
master_dict_eighth_row = fill_master_dict(6771, 6779, DataType.ni, 400, "/IPTS-34969/images/tpx3/raw/df/February24_2025_Ni_powder_5_0C_MCP_TPX3_2_0AngsMin/")
master_dict.update(master_dict_eighth_row)


In [9]:
retrieve_metadata(master_dict)

In [ ]:
# master_dict[6690]

{'data_type': 'ob',
 'l_over_d': 400,
 'data_path': '/IPTS-34969/images/tpx3/raw/radiographs/February20_2025_Ni_5C_MCP_TPX3_0_7AngsMin/',
 'nexus_path': '/SNS/VENUS/IPTS-34969/nexus/VENUS_6690.nxs.h5',
 'start_time': '2025-02-20T10:57:57.493248667-05:00',
 'start_time_friendly_display': '2/20 10:57:57',
 'start_time_in_seconds': 1740067077,
 'BL10:Det:BM1:Histo:Daq:Rate': {'mean': 531.1718810351487,
  'min': 160,
  'max': 755},
 'BL10:Det:T1:CountRate_RBV': {'mean': 3906921.0412802277,
  'min': 0,
  'max': 7676246},
 'BL10:CS:tpx3:cam1:PelEvtRate_RBV': {'mean': 3911909.8164803744,
  'min': 2394090,
  'max': 4050313}}

# combine results

In [ ]:
# bm_averaged_count_rate = []
# bm_averaged_count_rate_error_low = []
# bm_averaged_count_rate_error_high = []

# averaged_count_rate_adted = []
# averaged_count_rate_adted_error_low = []
# averaged_count_rate_adted_error_high = []

# averaged_count_rate_serval = []
# averaged_count_rate_serval_error_low = []
# averaged_count_rate_serval_error_high = []

# nexus_runs = []
# acquisition_time_friendly = []
# acquisition_time_in_seconds = []

# first_entry = master_dict[list_runs_for_plot[0]]
# plot_title = f"Average Count Rates - {first_entry[MasterKeys.data_type]} - L/D {first_entry[MasterKeys.l_over_d]}"

# for key in list_runs_for_plot:
#     bm_averaged_count_rate.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.mean])
#     bm_averaged_count_rate_error_low.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.mean] - master_dict[key][PV.bm_averaged_count_rate][PVtags.min])
#     bm_averaged_count_rate_error_high.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.max] - master_dict[key][PV.bm_averaged_count_rate][PVtags.mean] )
    
#     averaged_count_rate_adted.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.mean])
#     averaged_count_rate_adted_error_low.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.mean] - master_dict[key][PV.averaged_count_rate_adted][PVtags.min])
#     averaged_count_rate_adted_error_high.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.max] - master_dict[key][PV.averaged_count_rate_adted][PVtags.mean])

#     averaged_count_rate_serval.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.mean])
#     averaged_count_rate_serval_error_low.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.mean] - master_dict[key][PV.averaged_count_rate_serval][PVtags.min])
#     averaged_count_rate_serval_error_high.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.max] - master_dict[key][PV.averaged_count_rate_serval][PVtags.mean])
# list_runs_for_plot = np.arange(6690, 6713)
#     nexus_runs.append(key)
#     acquisition_time_in_seconds.append(master_dict[key][MasterKeys.start_time_in_seconds])
#     acquisition_time_friendly.append(master_dict[key][MasterKeys.start_time_friendly_display])

# my_dict = {PV.bm_averaged_count_rate: bm_averaged_count_rate,
#         PV.bm_averaged_count_rate + "_error_low": bm_averaged_count_rate_error_low,
#         PV.bm_averaged_count_rate + "_error_high": bm_averaged_count_rate_error_high,
#         PV.averaged_count_rate_adted: averaged_count_rate_adted,
#         PV.averaged_count_rate_adted + "_error_low": averaged_count_rate_adted_error_low,
#         PV.averaged_count_rate_adted + "_error_high": averaged_count_rate_adted_error_high,
#         PV.averaged_count_rate_serval: averaged_count_rate_serval,
#         PV.averaged_count_rate_serval + "_error_low": averaged_count_rate_adted_error_low,
#         PV.averaged_count_rate_serval + "_error_high": averaged_count_rate_adted_error_high,
#         "nexus_runs": nexus_runs,
#         MasterKeys.start_time_friendly_display: acquisition_time_friendly,
#         MasterKeys.start_time_in_seconds: acquisition_time_in_seconds}
# df = pd.DataFrame(my_dict)



In [77]:
def display_plots(from_run, to_run, master_dict):

    list_runs_for_plot = np.arange(from_run, to_run+1)

    bm_averaged_count_rate = []
    bm_averaged_count_rate_error_low = []
    bm_averaged_count_rate_error_high = []

    averaged_count_rate_adted = []
    averaged_count_rate_adted_error_low = []
    averaged_count_rate_adted_error_high = []

    averaged_count_rate_serval = []
    averaged_count_rate_serval_error_low = []
    averaged_count_rate_serval_error_high = []

    nexus_runs = []
    acquisition_time_friendly = []
    acquisition_time_in_seconds = []

    first_entry = master_dict[list_runs_for_plot[0]]
    plot_title = f"Average Count Rates - {first_entry[MasterKeys.data_type]} - L/D {first_entry[MasterKeys.l_over_d]}"

    for key in list_runs_for_plot:
        bm_averaged_count_rate.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.mean])
        bm_averaged_count_rate_error_low.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.mean] - master_dict[key][PV.bm_averaged_count_rate][PVtags.min])
        bm_averaged_count_rate_error_high.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.max] - master_dict[key][PV.bm_averaged_count_rate][PVtags.mean] )
        
        averaged_count_rate_adted.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.mean])
        averaged_count_rate_adted_error_low.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.mean] - master_dict[key][PV.averaged_count_rate_adted][PVtags.min])
        averaged_count_rate_adted_error_high.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.max] - master_dict[key][PV.averaged_count_rate_adted][PVtags.mean])

        averaged_count_rate_serval.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.mean])
        averaged_count_rate_serval_error_low.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.mean] - master_dict[key][PV.averaged_count_rate_serval][PVtags.min])
        averaged_count_rate_serval_error_high.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.max] - master_dict[key][PV.averaged_count_rate_serval][PVtags.mean])

        nexus_runs.append(key)
        acquisition_time_in_seconds.append(master_dict[key][MasterKeys.start_time_in_seconds])
        acquisition_time_friendly.append(master_dict[key][MasterKeys.start_time_friendly_display])

    my_dict = {PV.bm_averaged_count_rate: bm_averaged_count_rate,
            PV.bm_averaged_count_rate + "_error_low": bm_averaged_count_rate_error_low,
            PV.bm_averaged_count_rate + "_error_high": bm_averaged_count_rate_error_high,
            PV.averaged_count_rate_adted: averaged_count_rate_adted,
            PV.averaged_count_rate_adted + "_error_low": averaged_count_rate_adted_error_low,
            PV.averaged_count_rate_adted + "_error_high": averaged_count_rate_adted_error_high,
            PV.averaged_count_rate_serval: averaged_count_rate_serval,
            PV.averaged_count_rate_serval + "_error_low": averaged_count_rate_adted_error_low,
            PV.averaged_count_rate_serval + "_error_high": averaged_count_rate_adted_error_high,
            "nexus_runs": nexus_runs,
            MasterKeys.start_time_friendly_display: acquisition_time_friendly,
            MasterKeys.start_time_in_seconds: acquisition_time_in_seconds}
    df = pd.DataFrame(my_dict)

    layout = go.Layout(
        height=800,
        xaxis=XAxis(
            tickmode='array',
            tickvals=df[MasterKeys.start_time_in_seconds],
            ticktext=df[MasterKeys.start_time_friendly_display],
            side='top',
            title='Date/Time'
        ),
        xaxis2=XAxis(
            tickmode='array',
            tickvals=df[MasterKeys.start_time_in_seconds],
            ticktext=df["nexus_runs"],
            title='NeXus',
            side='bottom'
        ),
        yaxis=YAxis(
            title="Average counts/s",
        ),
        yaxis2=YAxis(
            title="Average counts/s",
        ),
        title={'text': plot_title,
            # 'xanchor': 'center',
            'y': 0.99,
            'yanchor': 'top',
            'font_color': 'red',
            # 'x': 0.5,
        },
    )

    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.01)

    fig.append_trace(go.Scatter(x=df[MasterKeys.start_time_in_seconds], 
                    y=df[PV.bm_averaged_count_rate], 
                    #  error_y=dict(type='data',
                    #               array=df[PV.bm_averaged_count_rate + "_error_high"],
                    #               arrayminus=df[PV.bm_averaged_count_rate + "_error_low"],
                    #               visible=True),
                    mode='lines+markers',
                    name=PVnames.bm_averaged_count_rate,
                    marker_color='red',
                    line=dict(color='red', width=1),
                    ),
                    row=1, col=1)

    fig.append_trace(go.Scatter(x=df[MasterKeys.start_time_in_seconds], 
                    y=df[PV.averaged_count_rate_adted], 
                    #  error_y=dict(type='data',
                    #               array=df[PV.averaged_count_rate_adted + "_error_high"],
                    #               arrayminus=df[PV.averaged_count_rate_adted + "_error_low"],
                    #               visible=True),
                    mode='lines+markers',
                    name=PVnames.averaged_count_rate_adted,
                    marker_color='green',
                    line=dict(color='green', width=1),
                    ),
                    row=2, col=1)

    fig.append_trace(go.Scatter(x=df[MasterKeys.start_time_in_seconds], 
                    y=df[PV.averaged_count_rate_serval], 
                    #  error_y=dict(type='data',
                    #               array=df[PV.averaged_count_rate_adted + "_error_high"],
                    #               arrayminus=df[PV.averaged_count_rate_adted + "_error_low"],
                    #               visible=True),
                    mode='lines+markers',
                    name=PVnames.averaged_count_rate_serval,
                    marker_color='violet',
                    line=dict(color='violet', width=1),
                    ),
                    row=2, col=1)

    fig.update_yaxes(type='log')
    fig.update_layout(layout)

    fig.show()

In [79]:
display_plots(6690, 6712, master_dict)
display_plots(6713, 6715, master_dict)
display_plots(6718, 6729, master_dict)
display_plots(6730, 6743, master_dict)
display_plots(6744, 6749, master_dict)
display_plots(6750, 6762, master_dict)
display_plots(6763, 6770, master_dict)
display_plots(6771, 6779, master_dict)


Other comparisons

In [112]:
def display_2_rows(group1=None, group2=None, label1=None, label2=None, master_dict=master_dict):

    group1_list_runs_for_plot = np.arange(group1[0], group1[1]+1)

    bm_averaged_count_rate = []
    bm_averaged_count_rate_error_low = []
    bm_averaged_count_rate_error_high = []

    averaged_count_rate_adted = []
    averaged_count_rate_adted_error_low = []
    averaged_count_rate_adted_error_high = []

    averaged_count_rate_serval = []
    averaged_count_rate_serval_error_low = []
    averaged_count_rate_serval_error_high = []

    nexus_runs = []
    acquisition_time_friendly = []
    acquisition_time_in_seconds = []

    first_entry = master_dict[group1_list_runs_for_plot[0]]
    plot_title = f"Average Count Rates - {first_entry[MasterKeys.data_type]} - L/D {first_entry[MasterKeys.l_over_d]}"

    for key in group1_list_runs_for_plot:
        bm_averaged_count_rate.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.mean])
        bm_averaged_count_rate_error_low.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.mean] - master_dict[key][PV.bm_averaged_count_rate][PVtags.min])
        bm_averaged_count_rate_error_high.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.max] - master_dict[key][PV.bm_averaged_count_rate][PVtags.mean] )
        
        averaged_count_rate_adted.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.mean])
        averaged_count_rate_adted_error_low.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.mean] - master_dict[key][PV.averaged_count_rate_adted][PVtags.min])
        averaged_count_rate_adted_error_high.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.max] - master_dict[key][PV.averaged_count_rate_adted][PVtags.mean])

        averaged_count_rate_serval.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.mean])
        averaged_count_rate_serval_error_low.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.mean] - master_dict[key][PV.averaged_count_rate_serval][PVtags.min])
        averaged_count_rate_serval_error_high.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.max] - master_dict[key][PV.averaged_count_rate_serval][PVtags.mean])

        nexus_runs.append(key)
        acquisition_time_in_seconds.append(master_dict[key][MasterKeys.start_time_in_seconds])
        acquisition_time_friendly.append(master_dict[key][MasterKeys.start_time_friendly_display])

    my_dict_group1 = {PV.bm_averaged_count_rate: bm_averaged_count_rate,
            PV.bm_averaged_count_rate + "_error_low": bm_averaged_count_rate_error_low,
            PV.bm_averaged_count_rate + "_error_high": bm_averaged_count_rate_error_high,
            PV.averaged_count_rate_adted: averaged_count_rate_adted,
            PV.averaged_count_rate_adted + "_error_low": averaged_count_rate_adted_error_low,
            PV.averaged_count_rate_adted + "_error_high": averaged_count_rate_adted_error_high,
            PV.averaged_count_rate_serval: averaged_count_rate_serval,
            PV.averaged_count_rate_serval + "_error_low": averaged_count_rate_adted_error_low,
            PV.averaged_count_rate_serval + "_error_high": averaged_count_rate_adted_error_high,
            "nexus_runs": nexus_runs,
            MasterKeys.start_time_friendly_display: acquisition_time_friendly,
            MasterKeys.start_time_in_seconds: acquisition_time_in_seconds}
    df_group1 = pd.DataFrame(my_dict_group1)

    group2_list_runs_for_plot = np.arange(group2[0], group2[1]+1)

    bm_averaged_count_rate = []
    bm_averaged_count_rate_error_low = []
    bm_averaged_count_rate_error_high = []

    averaged_count_rate_adted = []
    averaged_count_rate_adted_error_low = []
    averaged_count_rate_adted_error_high = []

    averaged_count_rate_serval = []
    averaged_count_rate_serval_error_low = []
    averaged_count_rate_serval_error_high = []

    nexus_runs = []
    acquisition_time_friendly = []
    acquisition_time_in_seconds = []

    first_entry = master_dict[group2_list_runs_for_plot[0]]
    plot_title = f"Average Count Rates - {first_entry[MasterKeys.data_type]} - L/D {first_entry[MasterKeys.l_over_d]}"

    for key in group2_list_runs_for_plot:
        bm_averaged_count_rate.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.mean])
        bm_averaged_count_rate_error_low.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.mean] - master_dict[key][PV.bm_averaged_count_rate][PVtags.min])
        bm_averaged_count_rate_error_high.append(master_dict[key][PV.bm_averaged_count_rate][PVtags.max] - master_dict[key][PV.bm_averaged_count_rate][PVtags.mean] )
        
        averaged_count_rate_adted.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.mean])
        averaged_count_rate_adted_error_low.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.mean] - master_dict[key][PV.averaged_count_rate_adted][PVtags.min])
        averaged_count_rate_adted_error_high.append(master_dict[key][PV.averaged_count_rate_adted][PVtags.max] - master_dict[key][PV.averaged_count_rate_adted][PVtags.mean])

        averaged_count_rate_serval.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.mean])
        averaged_count_rate_serval_error_low.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.mean] - master_dict[key][PV.averaged_count_rate_serval][PVtags.min])
        averaged_count_rate_serval_error_high.append(master_dict[key][PV.averaged_count_rate_serval][PVtags.max] - master_dict[key][PV.averaged_count_rate_serval][PVtags.mean])

        nexus_runs.append(key)
        acquisition_time_in_seconds.append(master_dict[key][MasterKeys.start_time_in_seconds])
        acquisition_time_friendly.append(master_dict[key][MasterKeys.start_time_friendly_display])

    my_dict_group2 = {PV.bm_averaged_count_rate: bm_averaged_count_rate,
            PV.bm_averaged_count_rate + "_error_low": bm_averaged_count_rate_error_low,
            PV.bm_averaged_count_rate + "_error_high": bm_averaged_count_rate_error_high,
            PV.averaged_count_rate_adted: averaged_count_rate_adted,
            PV.averaged_count_rate_adted + "_error_low": averaged_count_rate_adted_error_low,
            PV.averaged_count_rate_adted + "_error_high": averaged_count_rate_adted_error_high,
            PV.averaged_count_rate_serval: averaged_count_rate_serval,
            PV.averaged_count_rate_serval + "_error_low": averaged_count_rate_adted_error_low,
            PV.averaged_count_rate_serval + "_error_high": averaged_count_rate_adted_error_high,
            "nexus_runs": nexus_runs,
            MasterKeys.start_time_friendly_display: acquisition_time_friendly,
            MasterKeys.start_time_in_seconds: acquisition_time_in_seconds}
    df_group2 = pd.DataFrame(my_dict_group2)


    layout = go.Layout(
        height=800,
        # xaxis=XAxis(
        #     tickmode='array',
        #     tickvals=df_group1[MasterKeys.start_time_in_seconds],
        #     ticktext=df[MasterKeys.start_time_friendly_display],
        #     side='top',
        #     title='Date/Time'
        # ),
        # xaxis2=XAxis(
        #     tickmode='array',
        #     tickvals=df[MasterKeys.start_time_in_seconds],
        #     ticktext=df["nexus_runs"],
        #     title='NeXus',
        #     side='bottom'
        # ),
        yaxis=YAxis(
            title="Average counts/s",
        ),
        yaxis2=YAxis(
            title="Average counts/s",
        ),
        title={'text': plot_title,
            # 'xanchor': 'center',
            'y': 0.99,
            'yanchor': 'top',
            'font_color': 'red',
            # 'x': 0.5,
        },
    )

    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.01)

    fig.append_trace(go.Scatter(y=df_group1[PV.bm_averaged_count_rate], 
                    mode='lines+markers',
                    name=PVnames.bm_averaged_count_rate + " - " + label1,
                    marker_color='red',
                    marker_symbol='star',
                    marker_size=10,
                    line=dict(color='red', width=1, dash='dash'),
                    ),
                    row=1, col=1)

    fig.append_trace(go.Scatter(y=df_group2[PV.bm_averaged_count_rate], 
                    mode='lines+markers',
                    name=PVnames.bm_averaged_count_rate + " - " + label2,
                    marker_color='blue',
                    marker_symbol='star',
                    marker_size=10,
                    line=dict(color='blue', width=1),
                    ),
                    row=1, col=1)

    fig.append_trace(go.Scatter(y=df_group1[PV.averaged_count_rate_adted], 
                    mode='lines+markers',
                    name=PVnames.averaged_count_rate_adted + " - " + label1,
                    marker_color='red',
                    marker_symbol='x',
                    marker_size=10,
                    line=dict(color='red', width=1, dash='dash'),
                    ),
                    row=2, col=1)

    fig.append_trace(go.Scatter(y=df_group2[PV.averaged_count_rate_adted], 
                    mode='lines+markers',
                    name=PVnames.averaged_count_rate_adted + " - " + label2,
                    marker_color='blue',
                    marker_symbol='x',
                    marker_size=10,
                    line=dict(color='blue', width=1, dash='dot' ),
                    ),
                    row=2, col=1)

    fig.append_trace(go.Scatter(y=df_group1[PV.averaged_count_rate_serval], 
                    mode='lines+markers',
                    name=PVnames.averaged_count_rate_serval + " - " + label1,
                    marker_color='red',
                    marker_symbol='diamond-tall',
                    marker_size=10,
                    line=dict(color='red', width=1, dash='dot'),
                    ),
                    row=2, col=1)

    fig.append_trace(go.Scatter(y=df_group2[PV.averaged_count_rate_serval], 
                    mode='lines+markers',
                    name=PVnames.averaged_count_rate_serval + " - " + label2,
                    marker_color='blue',
                    marker_symbol='diamond-tall',
                    marker_size=10,
                    line=dict(color='blue', width=1),
                    ),
                    row=2, col=1)

    fig.update_yaxes(type='log')
    fig.update_layout(layout)

    fig.show()





In [113]:
# plot of OB, L/D 400  row #6 and row #13 
display_2_rows(group1=[6690, 6712], group2=[6763, 6770], label1="row #6", label2="row #13", master_dict=master_dict)

In [114]:
# plot of Ni, L/D 400 row #8 and row #14
display_2_rows(group1=[6718, 6729], group2=[6771, 6779], label1="row #8", label2="row #14", master_dict=master_dict)

In [115]:
# plot of Ni, L/D 800 row #10 and row #11
display_2_rows(group1=[6730, 6743], group2=[6744, 6749], label1="row #10", label2="row #11", master_dict=master_dict)